In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from underthesea import word_tokenize
import joblib
import pandas as pd
import numpy as np
import regex as re


def load_url(url):
    driver=webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    time.sleep(3)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)
    num_rate=driver.execute_script("var count = document.querySelectorAll('[data-testid=post_message]').length; return count")

    csvdata = []
    for i in range (0,num_rate):
        script="var post= document.querySelectorAll('[data-testid=post_message]')["+str(i)+"].innerText; return post"
        post=driver.execute_script(script)
        csvdata.append(post)
    driver.close()
    return csvdata

def remove_icon(text):
    pattern = re.compile('[\W_]+')
    dirty_list = text.split()
    clean_list = [pattern.sub('', word) for word in dirty_list]
    clean_string = ' '.join(clean_list)
    return clean_string

def standardize_data(post):
    post=remove_icon(post)
    post=post.replace('.','')
    post=post.replace(',','')
    post=post.replace(' k ',' không ')
    post=post.replace(' mk ',' mình ')
    post=post.replace('\n',' ')
    post=post.replace(' cx ',' cũng ')
    post=post.replace('  ',' ')
    post=post.replace(' r ',' rồi ')
    post=post.replace(' dc ',' được ')
    post=post.strip()
    post = re.sub(r"[\.,\?]+$-", "", post)
    post=word_tokenize(post.lower(),format="text")
    return post

def tokenizer(row):
    return word_tokenize(row, format="text")


In [2]:
url =  'https://www.facebook.com/pg/ilovelotteriadongda/reviews/'
data = load_url(url)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\hungn\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [3]:
data

['Thực đơn ngon miệng, nhân viên phục vụ tốt',
 'Phục vụ k tốt đồ ăn không chất lượng . Tôi rất thất vọng với trải nhiệm lần này',
 'Đồ ăn rất ngon, nhân viên phục vụ rất nhiệt tình',
 'Thích combo 88k ở đây nhất!😁',
 'Mình đến ăn gà với bạn bè để có thể vui chơi giải trí nhưng mà đến quán toàn phải ăn cơm 🐶 cũng đủ no r, đã thế nhạc toàn là nhạc tình cảm thôi làm cho 1 người cô đơn như mình cảm thấy rất tủi.\nMong lần sau quán cho các cặp đôi yêu nhau chia đôi bàn để mình cảm thấy vui 😢',
 'Quá là điều bình thường nuôn. Chả có gì đặc sắc, đồ ăn thì max. Ăn 1 bữa chắc đau ví cả tháng🤣🤣🤣',
 'Hay mua theo combo cho tiện nhưngg không được enjoy cho lắm 😀 Không có gì đặc sắc',
 'dịch vụ giao hàng tồi , ồn ào',
 'Lâu rùi k ra ngoài đường quên luôn mùi Lotteria. Đôi lời gửi gắm, mk thích ăn kem ốc quế mà lúc nào đến cũng hết vỏ quế à 😰😰, mong sẽ nhập hàng liên tục ngày nào cx sẽ ra ăn 👍',
 'nhân viên phục vụ không tốt',
 'Đồ ăn khá ngon. combo có 158 mà nhiều đồ gheee',
 'Ăn được ăn ok ăn n

In [4]:
datafr=pd.DataFrame(data)
datafr[0]=datafr[0].apply(standardize_data)
datafr[0]=datafr[0].apply(tokenizer)

In [5]:
# Embedding
X_val = datafr[0]
emb = joblib.load('tfidf.pkl')
X_val = emb.transform(X_val)
#  Predict
model = joblib.load('saved_model.pkl')
result = model.predict(X_val)
result

array([1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1], dtype=int64)

In [6]:
datafr['Sentiment']=result
datafr

,0,Sentiment
0,thực_đơn ngon_miệng nhân_viên phục_vụ tốt,1
1,phục_vụ không tốt đồ ăn không chất_lượng tôi r...,0
2,đồ ăn rất ngon nhân_viên phục_vụ rất nhiệt_tình,1
3,thích combo 88 k ở đây nhất,1
4,mình đến ăn gà với bạn_bè để có_thể vui_chơi g...,1
5,quá là điều bình_thường nuôn chả có gì đặc_sắc...,0
6,hay mua theo combo cho tiện nhưngg không được ...,0
7,dịch_vụ giao hàng tồi ồn_ào,0
8,lâu rùi không ra ngoài đường quên luôn mùi lot...,1
9,nhân_viên phục_vụ không tốt,0
